In [2]:
import requests
from bs4 import BeautifulSoup
import urllib3
import html
import re
import json
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from office365.sharepoint.lists.list import List
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import UserCredential
from office365.sharepoint.files.file import File


In [ ]:
# API Query with pagination control (ability to iterate through all pages/records)

import csv
import requests
import time

# API base URL and static parameters
api_url = "https://my.intelligence2day.com/components/api/search.cfc"
params = {
    "method": "query",
    "APIid": "I2DE_4880557FFC6ABA165C916880849F9CAC",
    "authKey": "c51e7492-ab7f-46d8-9d10-edd4e434d2c1",
    "customerGUID": "b6150206-d9b1-4963-8907-22b7695c0477",
    "accessGroups": "8329",
    "queryString": "*:*",  # Query for all records
    "returnFields": "title,summary,attachmenturl",  # Fields to return
    "maxRows": 100,  # Limit to 10 results
    "sort": "uid desc",  # Sort by UID in descending order
}

# Pagination control
cursor = "*"  # Start with an empty cursor for the first request
has_more = True
total_articles = 0
page = 1
all_articles = []  # To store all article data

while has_more:
    time.sleep(2)
    print(f"\n--- Fetching Page {page} ---")

    # Update the cursor in the request parameters for pagination
    params["cursorMark"] = cursor

    # Make the request
    response = requests.get(api_url, params=params, verify=False)

    # Print the status code
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        try:
            data = response.json()  # Parse the response as JSON
            print("Returned Data:")
            print(data)  # Print the raw JSON response
            
            articles = data.get("docs", [])
            next_cursor = data.get("nextCursormark", None)

            if not articles:
                print("No more articles returned.")
                break

            print(f"Retrieved {len(articles)} articles on page {page}.")

            # Print the articles' title, summary, and URL
            for i, article in enumerate(articles, 1):
                title = article.get("headline", "No title")
                summary = article.get("summary", "No summary")
                url = article.get("url", "No URL")

                all_articles.append({"title": title, "summary": summary, "url": url})

                print(f"\nArticle {total_articles + i}")
                print(f"Title   : {title}")
                print(f"Summary : {summary}")
                print(f"URL     : {url}")

            total_articles += len(articles)
            page += 1

            # Prepare for the next iteration with the nextCursormark
            if next_cursor:
                cursor = next_cursor  # Update the cursor for the next request
            else:
                has_more = False  # No more pages, end the loop

        except ValueError:
            print("Error: Response is not valid JSON.")
            break
    else:
        print(f"Request failed with status code {response.status_code}")
        break

print(f"\n✅ Total articles fetched: {total_articles}")

# Save to CSV
#with open("articles.csv", mode="w", newline="", encoding="utf-8") as file:
#    writer = csv.DictWriter(file, fieldnames=["title", "summary", "url"])
#    writer.writeheader()
#    writer.writerows(all_articles)


In [12]:
# API Query to test functionality (without pagination)


# API base URL and static parameters
api_url = "https://my.intelligence2day.com/components/api/search.cfc"
params = {
    "method": "query",
    "APIid": "I2DE_4880557FFC6ABA165C916880849F9CAC",
    "authKey": "c51e7492-ab7f-46d8-9d10-edd4e434d2c1",
    "customerGUID": "b6150206-d9b1-4963-8907-22b7695c0477",
    "accessGroups": "8329",
    "queryString": "dateline:[NOW-2DAYS TO NOW]",  # Query for all records within time range
    #"queryString": "dateline:[NOW-1MONTH TO NOW]",  # Query for all records within time range
    #"queryString": "dateline:[{last_run_date} TO NOW]", # Query for all records between the last run date (max run date in excel) and now
    "maxRows": 500,  # Limit to 10 results
    "sort": "dateline desc",  # Sort by UID in descending order

}

total_articles = 0
all_articles = []  # To store all article data


# Make the request
response = requests.get(api_url, params=params, verify=False)

# Print the status code
print(f"Status Code: {response.status_code}")
    
if response.status_code == 200:
    try:
        data = response.json()  # Parse the response as JSON
        print("Returned Data:")
        print(data)  # Print the raw JSON response
            
        articles = data.get("docs", [])


        if not articles:
            print("No more articles returned.")
            

        print(f"Retrieved {len(articles)} articles")

        # Print the articles' title, summary, and URL
        for i, article in enumerate(articles, 1):
            title = article.get("headline", "No title")
            summary = article.get("summary", "No summary")
            url = article.get("url", "No URL")
            date = article.get("dateline", "No date")

            all_articles.append({"title": title, "summary": summary, "url": url})

            print(f"\nArticle {total_articles + i}")
            print(f"Title   : {title}")
            print(f"Summary : {summary}")
            print(f"URL     : {url}")
            print(f"Date    : {date}")


    except ValueError:
        print("Error: Response is not valid JSON.")
        
else:
    print(f"Request failed with status code {response.status_code}")
    
total_articles = len(all_articles)
print(f"\n✅ Total articles fetched: {total_articles}")



Status Code: 200
Returned Data:
{'numFound': 125, 'start': 0, 'docs': [{'uid': '34229168', 'url': 'https://my.intelligence2day.com/cc/view/article/?a=8cc6312d1ea200f27dd58b2ee100aba9', 'dateline': '2025-05-09T06:04:11Z', 'headline': 'Copeland to Be Stronger with Right Investments in Right Areas', 'summary': ' In May 2023, Copeland, a global provider of sustainable climate solutions in the heating, ventilation, air conditioning, and refrigeration (HVAC&amp;R) industry, separated from Emerson and became a standalone business.'}, {'uid': '34227057', 'url': 'https://my.intelligence2day.com/cc/view/article/?a=90f68ef3d4e711738dfd4020056255dd', 'dateline': '2025-05-09T05:02:31Z', 'headline': 'BALM stellt neuartige Lkw-Waage auf der A4 vor', 'summary': 'Besserer Schutz von Infrastruktur und ein Plus für m mehr Verkehrssicherheit: Bundesamts für Logistik und Mobilität (BALM) stellt neuartiges Wiegesystem auf der A4 bei Düren vor. Das kann die Lösung.'}, {'uid': '34226468', 'url': 'https://my.i